# eCallisto Validation

The aim of this package is to:

1) Append the Metadata into the datebase (save_to_sql.py).
* Metadata is defined as the data providing information about one or more aspects of the data; it is used to summarize basic information about data which can make tracking and working with specific data easier. 
* Database is an organized collection of structured information, or data, typically stored electronically in a computer system.

2) Calculate the standard deviation and update it into the datebase (STD.py).
* What is the standard deviation? The standard deviation is the average amount of variability in your data set. It tells you, on average, how far each score lies from the mean. In normal distributions, a high standard deviation means that values are generally far from the mean, while a low standard deviation indicates that values are clustered close to the mean.

3) Choose 10 Spectrograms pro Station and plot them with 4 columns, then save them as a pdf file (Testing_10000.py).


### Importing the Modules 

In [ ]:
import os
import sys
import io
import glob
import astropy.io.fits

import numpy as np
import time
import psycopg2
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

from eCallistoProject import plot_config

module_path = os.path.abspath(os.path.join('radiospectra'))
if module_path not in sys.path:
    sys.path.append(module_path)


import radiospectra
from radiospectra.sources import CallistoSpectrogram
from matplotlib.backends.backend_pdf import PdfPages

import warnings
warnings.filterwarnings("ignore")

* The path to the data in my notebook

In [ ]:
PATH = 'R:\\radio\\2002-20yy_Callisto\\2017\\09'

## save_to_sql.py

* Converting the time (by Kushtrim).

In [ ]:
def __to_timestamp(date_string, time_string):
    
                sixty_seconds = int(time_string[6:8]) == 60
                sixty_minutes = int(time_string[3:5]) == 60
                twentyfour_hours = int(time_string[:2]) == 24
                
                # replacing  24 to 00 
                if sixty_seconds :
                    time_string = time_string[:6] + '59' + time_string[8:]
                if sixty_minutes :
                    time_string = time_string[:3] + '59' + time_string[5:]
                if twentyfour_hours :
                    time_string = '23' + time_string[2:]
                if re.findall("\.\d+", time_string):
                    time_string = time_string[:-4]
                    
                # lost time     
                ts = datetime.datetime.strptime(
                    '%s %s' % (date_string, time_string), '%Y/%m/%d %H:%M:%S')                  
                ts += datetime.timedelta(hours = int(twentyfour_hours),
                                         minutes = int(sixty_minutes),
                                         seconds = int(sixty_seconds))
                
                return ts  

* Establish a DBAPI connection at localhost:5432 when a connection request


In [ ]:
engine = create_engine("postgresql+psycopg2://" + 'postgres' + ":" + 'ecallistohackorange'
                        + "@" + 'localhost' + "/" + 'validation')

### The aim of this Script to:
* Walk through the path to find the data.
* Open the fits-file from the header list (hdulist).
* Call the Mata Data from the header list.

In [ ]:
for root, dirs, files in os.walk(PATH):
    for file in files:
        if file.endswith('.fit.gz'):
                
            full_path = os.path.join(root, file)

            hdulist = astropy.io.fits.open(full_path)

            instrument_name = hdulist[0].header['INSTRUME'] 
            date_obs = hdulist[0].header['DATE-OBS'] 
            time_obs = hdulist[0].header['TIME-OBS']
            date_end = hdulist[0].header['DATE-END'] 
            time_end = hdulist[0].header['TIME-END'] 

* Combine date and time obs, date and time end.


In [ ]:
start_time = __to_timestamp(date_obs, time_obs)
end_time = __to_timestamp(date_end, time_end)

### In this Script we are:
* Creating a dataframe in pandas.
* Inserting the data into the Dataframe.

In [ ]:
"""
- creating dataframe in pandas
- Inserting the data into the DataFrame
"""
data={
    'path':[full_path],
    'file_name':[file],
    'instrument_name':[instrument_name],
    'start_time':[start_time] ,
    'end_time':[end_time],
    'std' : [None]
      }

data_frame = pd.DataFrame(data, index = [df]) 

* Create a connection between pandas and sql and then append the dataframe into sql.

In [ ]:
data_frame.to_sql('ecallisto', con = engine, if_exists = 'append',
                  chunksize = 50000, index=False)

## STD.py (Standard Deviation)

* Connect to the database Postgres.

In [ ]:
connection = psycopg2.connect(user="postgres",
                              password="ecallistohackorange",
                              host="localhost",
                              port="5432",
                              database="validation")
cursor = connection.cursor()

* Select from the table "ecallisto" from Database to calculate the Std.

In [ ]:
"""Return a table from DB where the std is null"""

cursor.execute("""SELECT * from  ecallisto WHERE std is null ORDER BY id""")

* Subtract the background.
* Calculate the std (standard deviation).
* Update the std into the Database ecallisto (the name of the database).
* Exception to catch all the errors and append them into List_of_err.
* Finally close the connection.

In [ ]:
"""
- [1] is the index of file_name in the cursor.
- subtract the background and then calculate the std.
- update the std into the Database ecallisto
- expetion to catch the all errors and append them into the List_of_err to check the erros.
- close the connection
"""

"""Return a list of STD and then update them into the DB"""

list_of_err =[] 
for path in cursor.fetchall():
    
    try:
        spec = CallistoSpectrogram.read(path[1])
        spec2 = spec.subtract_bg("subtract_bg_sliding_window", window_width=800, affected_width=1,
                                 amount=0.05, change_points=True)

        spec_std = spec2.data.std()
        sql_update_query = f"""UPDATE ecallisto SET std = {spec_std} where id = {path[0]} """
        cursor.execute(sql_update_query)
        connection.commit()
        
    except Exception as err:
        exception_type = type(err).__name__ + path[2]
        list_of_err.append(exception_type)
        
    finally:
        connection.close()
        print("Table after Updating the records")

## Testing_10'000.py

* Selecting 10 Spectrograms pro Station. 

In [ ]:
cursor.execute("""select * from (
                                 select ROW_NUMBER() OVER (partition by instrument_name order by id)
                                 as row_num, ecallisto.* FROM ecallisto
                                 ) t
                                 where row_num <=10
                                 order by instrument_name""")

To move the Axes:
* Source: https://gist.github.com/salotz/8b4542d7fe9ea3e2eacc1a2eef2532c5 
* By Kushtrim.

In [ ]:
def move_axes(fig, ax_source, ax_target):
    old_fig = ax_source.figure
    ax_source.remove()
    ax_source.figure = fig
    ax_source.set_ylabel('')
    ax_source.set_xlabel('')
        
    ax_source.set_position(ax_target.get_position())
    ax_target.remove()
    ax_target.set_aspect("equal")
    fig.axes.append(ax_source)
    fig.add_subplot(ax_source)
    
    plt.close(old_fig)

* To generate the Colormap (by Kushtrim).

In [ ]:
my_colormap = matplotlib.colors.LinearSegmentedColormap.from_list("myColorMap", plot_config.COLORMAP / 255)

* The aim of this script is to create a pdf file with several pages, as well as adding the plots and histograms to pdf files.

In [ ]:
with PdfPages('C:\\Users\\delbe\\OneDrive\\Desktop\\eCallisto_validation\\Plot_PDF.pdf') as pdf:
        pdf.savefig(fig_target)
        plt.close()

* The first column include the original data (Spectrogram).

In [ ]:
full_path = os.path.join(PATH, file[2])
spec = CallistoSpectrogram.read(full_path)
fig1, axs1 = plt.subplots(1, 4, figsize=(25,5))
ax1 = spec.plot(cmap=my_colormap, colorbar=None)
ax1.title.set_text("Original Data")
plt.close()

* The second column include the "Background subtracted" by Kushtrim.

In [ ]:
spec2 = spec.subtract_bg("constbacksub", "elimwrongchannels")
fig2 = plt.subplots(1, 4, figsize=(25,5))
ax2 = spec2.plot(cmap=my_colormap, colorbar=None, vmin=-5, vmax=5)
ax2.title.set_text("Background subtracted")
ax2.set_xlabel('Time[UT]')
ax2.set_ylabel('Frequency[MHz]')
plt.close()

* The third Column include the 'Gliding background subtracted' by Simon.

In [ ]:
spec3 = spec.subtract_bg("subtract_bg_sliding_window", window_width=800, affected_width=1,
                         amount=0.05, change_points=True)
fig3 = plt.figure(figsize=(25,5))
ax3 = spec3.plot(cmap=my_colormap, colorbar=None, vmin=-5, vmax=5)
ax3.title.set_text("Gliding background subtracted (window=800)")
ax3.set_xlabel('Time[UT]')
ax3.set_ylabel('Frequency[MHz]')
plt.close()

* The fourth column include the Histograms of "Background subtracted" and "Gliding background subtracted (window=800)"

In [ ]:
"""
# the histogram for Background subtracted.
# the histogram for Gliding background subtracted (window=800).
"""
data_hist3 = np.absolute(spec2.data.flatten())
data_hist4 = np.absolute(spec3.data.flatten())

fig4, ax4 = plt.subplots(figsize=(25,5))
ax4.hist(data_hist3 ,histtype='step',range= (0, 10), bins= 40, label='Bg_cbs_rfi')
ax4.hist(data_hist4 ,histtype='step',range= (0, 10), bins= 40, label='Bg_sub_sliding_rfi')

ax4.title.set_text("Histograms")
ax4.set_xlabel('Pixel value')
ax4.set_ylabel('Number of pixels')
plt.legend()
plt.close()

* Plot final plot by moving axes to the figure.

In [ ]:
fig_target, (axA, axB, axC, axD) = plt.subplots(1, 4, figsize=(30,5))
plt.suptitle(fig1._suptitle.get_text())

* Move the axes by Kushtrim.

In [ ]:
move_axes(fig_target, ax1, axA)
move_axes(fig_target, ax2, axB)
move_axes(fig_target, ax3, axC)
move_axes(fig_target, ax4, axD)
plt.show()